In [14]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from copy import deepcopy
import pandas as pd

import cv2
import csv
from natsort import natsorted
from surgeon_recording.synchronization import Synchro

from moviepy.editor import *

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import moviepy.video.io.ImageSequenceClip

import shutil
import math


In [15]:
synchro = Synchro()

# Manual entries 

## Manual entries (NEW RECORDER)

In [35]:
################################## CHOOSE WHICH RECORDING YOU WANT ###################################
recording_session = '70622'
subject = '1/1'

################################## ADAPT THE FPS IF MODIFIED #########################################
# SFITS set-up:
fps_gopro = 59.940   
fps_rs = 30
fps_micro = 30       

# Test at LASA: 
# "gopro" = used a phone (oneplus), 30fps (ATTENTION need to set resolution for this phone videos, but not if use gopro)
# "microscope" = used a gropro  (so 59.940 fps)


sensors = ['optitrack', 'emg', 'TPS_calibrated']

path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                 #lasa computer
#path_to_data_folder = '/Users/anael/Documents/GitHub/surgeon_recordings/data'   #windows computer

cameras = ['GOPRO', 'GOPRO2', 'GOPRO3', 'MICROSCOPE', 'REALSENSE']
folder = ['complete','segments']

# create automatically the folders we need to store what we create
folder_creation_all = ['CSV', 'SEGMENTATION', 'complete', 'segments']
folder_creation_rs = ['PNG', 'BAG']
folder_creation_global = ['ALL_VIEWS']

for camera in cameras:
    for folder1 in folder_creation_all:
        if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , folder1)):
            os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, folder1))
            
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , 'CSV', 'complete')):
        os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, 'CSV', 'complete'))
        
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , 'CSV', 'segments')):
        os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, 'CSV', 'segments'))
    
    if camera == 'REALSENSE':
        for folder2 in folder_creation_rs:
            if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , folder2)):
                os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, folder2))
    
if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'ALL_VIEWS')):
    os.makedirs(join(path_to_data_folder, recording_session , subject, 'ALL_VIEWS'))

if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SENSORS')):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SENSORS'))
        
if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION')):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION'))

if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SEGMENTATION_CSV')):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SEGMENTATION_CSV'))
        
fps = {
    "GOPRO": fps_gopro,
    "GOPRO2": fps_gopro,
    "GOPRO3": fps_gopro,
    "MICROSCOPE": fps_micro,
    "REALSENSE": fps_rs
}

data_folder = join('..', 'data_exp', recording_session , subject)

## Manual entries (for tests, old recorder = not same data format output for the csv sensors files and not same experimental data structure)

In [ ]:
"""######################## examples of tests definition ##########################
#recording_session = 'SFITS_28_03_22'
#subject = 'MAXIME'
#fps_gopro = 59.940
#fps_rs = 30
#fps_micro = 30
#sensors = []

recording_session = 'test240522'
subject = '1/01'
fps_gopro = 59.940   
fps_rs = 30
fps_micro = 59.940       
sensors = ['optitrack', 'emg', 'TPS_calibrated']

###########################################################################
path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                 #lasa computer
#path_to_data_folder = '/Users/anael/Documents/GitHub/surgeon_recordings/data'   #windows computer

cameras = ['GOPRO', 'GOPRO2', 'MICROSCOPE', 'REALSENSE']
folder = ['complete','segments']

folder_creation_all = ['CSV', 'SEGMENTATION', 'complete', 'segments']
folder_creation_rs = ['MP4', 'PNG', 'BAG']
folder_creation_global = ['ALL_VIEWS']

for camera in cameras:
    for folder1 in folder_creation_all:
        if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , folder1)):
            os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, folder1))
            
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , 'CSV', 'complete')):
        os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, 'CSV', 'complete'))
        
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , 'CSV', 'segments')):
        os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, 'CSV', 'segments'))
    
    if camera == 'REALSENSE':
        for folder2 in folder_creation_rs:
            if not os.path.exists(join(path_to_data_folder, recording_session , subject, camera , folder2)):
                os.makedirs(join(path_to_data_folder, recording_session , subject,  camera, folder2))
        
    
if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'ALL_VIEWS')):
    os.makedirs(join(path_to_data_folder, recording_session , subject, 'ALL_VIEWS'))
    
            
if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SEGMENTATION_CSV')):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SEGMENTATION_CSV'))
        
fps = {
    "GOPRO": fps_gopro,
    "GOPRO2": fps_gopro,
    "MICROSCOPE": fps_micro,
    "REALSENSE": fps_rs
}

data_folder = join('..', 'data_exp', recording_session , subject)"""

# PART 1: synchro + video conversion

## Convert video

### get segments into one video (gopro, microscope) + convert type, even if only one segment useful for the rest


In [ ]:
# convert microscope and gopro mp4 video
for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject)
    
    for camera in cameras:
        L =[]
        if camera != 'REALSENSE':
        
            for root, dirs, files in os.walk(join(data_folder, camera, 'segments')):
                files = natsorted(files)
                print(files)
                for file in files:
                    if os.path.splitext(file)[1] == '.mp4' or os.path.splitext(file)[1] == '.MP4':
                        filePath = os.path.join(root, file)
                        video = VideoFileClip(filePath)
                        #video = VideoFileClip(filePath, target_resolution=(1920, 1080))              # need to set manually resolution for oneplus videos
                        L.append(video)
                tmp = 'output' + camera + '.mp4'
                output_path = join(data_folder, camera, 'complete', tmp)
                final_clip = concatenate_videoclips(L)
                final_clip.write_videofile(output_path, fps=fps[camera], remove_temp=True, audio = False)



### bag to png to mp4

In [ ]:
# extract the png with the converter (from shell) to the folder PNG in the folder REALSENSE in the SUBJECT folder
for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject)
    camera = 'REALSENSE'

    bag_file = [x[2] for x in os.walk(join(data_folder, camera, 'BAG'))]
    print(bag_file[0][0])

    path_bag_file = join(path_to_data_folder, recording_session , subject, camera, 'BAG', bag_file[0][0])
    print(path_bag_file)

    path_png=  join(path_to_data_folder, recording_session , subject,  camera, 'PNG', 'a')
    print(path_png)
    os.system('rs-convert -i ' + path_bag_file + ' -p ' + path_png)
    
    image_folder=join(data_folder, camera, 'PNG')
    image_files = [os.path.join(image_folder,img) for img in os.listdir(image_folder) if img.endswith(".png")]

    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(natsorted(image_files), fps=fps[camera])
    path_mp4 = join(data_folder, camera, 'complete', 'rs_converted.mp4')
    print('mp4 file: ' + path_mp4)
    clip.write_videofile(path_mp4)

# PART 2 : Write CSV files

## Read the synchronization frames put manually in a csv file

In [ ]:
# read synchro frames from csv

file = 'synchro_frame.csv'
directory = join(data_folder, file)

data = pd.read_csv(directory)
rs_frame = int(data['rs_frame'])
micro_frame = int(data['micro_frame'])
gopro_frame = int(data['gopro_frame'])
gopro2_frame = int(data['gopro2_frame'])
gopro3_frame = int(data['gopro3_frame'])

frame = {
    "REALSENSE": rs_frame,
    "GOPRO": gopro_frame,
    "GOPRO2": gopro2_frame,
    "GOPRO3": gopro3_frame,
    "MICROSCOPE": micro_frame,
    
}


## Get absolute time from png names


In [37]:
# file directory to access the png files (Attention: also in the function to get video from png)
# the png exported from the bag file have the absolute timestamps as names

camera = 'REALSENSE'
image_folder=join(data_folder, camera, 'PNG')
# get all png images created
image_files = [os.path.join(image_folder,img) for img in os.listdir(image_folder) if img.endswith(".png")]

# get nb of frames of the rs camera
rs_nb_frame = len(image_files)

# get absolute time vector of rs from png files names
abs_time_vector = {}
abs_time_vector[camera] = [name[-32:-4] for name in natsorted(image_files)]

# get the absolute time corresponding to the synchro fram for RS camera
rs_abs_time = float(abs_time_vector['REALSENSE'][rs_frame])

## Get nb of frames

In [38]:
# get the nb of frames for each segments and the total videos for the gopro and microscope

nb_frames = {}
sum_frame = {}

for k in range(len(cameras)):    #for the different cameras
    camera = cameras[k]
    sum_frame[camera] = {}
    for j in range(len(folder)):  #for each segments and the complete video
        #get the name of the files of the videos
        files = [x[2] for x in os.walk(join(data_folder, camera,folder[j]))]   
        tmp = 0
        # for realsense: already have the total number of frames and there is only the complete video
        if camera == 'REALSENSE':
                    nb_frames[camera] = rs_nb_frame 
        # for the other cameras: check that the segments were put together correctly to form the new video = all the frames were put together
        else:
            if len(files) != 0:                                                    
                for i in range(len(files[0])):
                    # take each video
                    file = files[0][i]                                            
                    if os.path.splitext(file)[1] == '.MP4' or os.path.splitext(file)[1] == '.mp4':   #check if it is indeed a video file
                        # get informatio non number of frames
                        cap = cv2.VideoCapture(join(data_folder, camera, folder[j], file))
                        nb_frames[file] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                     # store nb of frames with file name as key
                        # if it is a segment: add the nb of frames of the segments
                        if folder[j] == 'segments':                                
                            tmp = tmp + nb_frames[file]
                            sum_frame[camera] = tmp
                
    # print the sum of frames of each segment
    print('Total nb of frames of segments for ', camera, sum_frame[camera])   
# print the nb of frame in the converted video
print('Nb of frames of the output GOPRO video:   ' , nb_frames['outputGOPRO.mp4'])
print('Nb of frames of the output MICROSCOPE video:   ', nb_frames['outputMICROSCOPE.mp4'])
print('Nb of frames of the output RS video:   ', nb_frames['REALSENSE'])  

# --> allows to check that we did not lose frames when converting videos and putting segments together
  

Total nb of frames of segments for  GOPRO 26037
Total nb of frames of segments for  GOPRO2 26028
Total nb of frames of segments for  GOPRO3 26157
Total nb of frames of segments for  MICROSCOPE 26494
Total nb of frames of segments for  REALSENSE {}
Nb of frames of the output GOPRO video:    26038
Nb of frames of the output MICROSCOPE video:    13245
Nb of frames of the output RS video:    808


## write CSV with absolute time for RS coming from PNG, and absolute time starting on the frame of the mvmt for the other, based on the abs time of RS

In [39]:
csv_names = []

for k in range(len(cameras)):
    camera = cameras[k]
    
    for j in range(len(folder)):
        files = [x[2] for x in os.walk(join(data_folder, camera ,folder[j]))]
        print(files)  
        
        # for RS camera: only the complete file (no segments)
        if camera == 'REALSENSE':
            
            rs_csv = join(data_folder,camera, 'CSV', 'complete', 'REALSENSE_abs_time.csv')
            f = open(rs_csv, 'w') 
            # create the csv writer
            writer = csv.writer(f)
            header = ['index','frequency','relative_time','abs_time']
            writer.writerow(header)
            rel_time = 0
            
            # for every frame: get the absolute time (from list of png names) and write a row with the nb of the frame and 
            # the corresponding relative time and absolute time
            for m in range((nb_frames[camera])):                            
                abs_time = abs_time_vector['REALSENSE'][m]
                row = [m,fps[camera],rel_time,float(abs_time)]               
                writer.writerow(row)
                
                #method 1: create relative time vector using the theoretical FPS of the camera
                rel_time = rel_time + 1/fps[camera]                      
                
                # method 2: create the relative time vector using the obtained delta t between the 2 absolute timestamps
                """if m < nb_frames[camera]-1:
                    #print(m)
                    rel_time = rel_time + (float(abs_time_vector['REALSENSE'][m+1])-float(abs_time_vector['REALSENSE'][m]))/1000
                    delta_t = (float(abs_time_vector['REALSENSE'][m+1])-float(abs_time_vector['REALSENSE'][m]))/1000
                else:
                    rel_time = rel_time + 1/fps[camera]
                    delta_t = 1"""
                # --> does not matter a lot as the RS is not exported for the segmentation afterwards (only used for the reference absolute time
                # at the synchronization frame)
                
            f.close()        
        # for the other cameras:
        else:
            for i in range(len(files[0])):
                file = files[0][i] 
                if os.path.splitext(file)[1] == '.mp4' or os.path.splitext(file)[1] == '.MP4': #check if video file
            
                    csv_names.append(join(data_folder,camera, 'CSV',folder[j], str(file)[0:-4] + '_abs_time.csv'))  #create the csv references
                    f = open(csv_names[-1], 'w')
                    # create the csv writer
                    writer = csv.writer(f)
                    header = ['index','frequency','relative_time','abs_time']
                    # write a row to the csv file
                    writer.writerow(header)
                    rel_time = 0
                    # the first absolute time we know is the one of the synchronization motion from the RS camera
                    abs_time = rs_abs_time                                        

                    #go over al the frames
                    for m in range((nb_frames[file])):
                        # if we are in the complete video AND we are after the synchronization frame: we start writing the absolute timestamps
                        # starting from the common reference and incrementing using the fps
                        if folder[j] == 'complete' and m >= frame[camera]:        
                            row = [m,fps[camera],rel_time,abs_time]               
                            abs_time = abs_time + 1/fps[camera]*1000 # the absolute time is in ms
                        else:
                        # else we are either before the synchro motion (no absolute time), or in the segments (not need the absolute time)
                            row = [m, fps[camera], rel_time]                      
                        writer.writerow(row)
                        
                        # we always increment the relative time using the therotical fps, starting from 0
                        rel_time = rel_time + 1/fps[camera]                       
           
                    f.close()
            
# the csv for the segments of the videos are built if we need to check the coherence between the files, but are not used afterwards

[['outputGOPRO.mp4']]
[['GOPR5875.MP4']]
[['outputGOPRO2.mp4']]
[['GH010910.MP4']]
[['outputGOPRO3.mp4']]
[['GH013739.MP4']]
[['outputMICROSCOPE.mp4']]
[['SegmentSDI20002.mp4', 'SegmentSDI20001.mp4']]
[['rs_converted.mp4']]
[[]]


# PART 3 : Segmentation

## read csv time data created to store them

In [40]:
#put back into a variable the saved csv data (time vectors)
# goal: can run the code from here if the videos were already treated and csv alreadz written, no need to run the first parts to store this data

index = {}
relative_time = {}
absolute_time = {}

for k in range(len(cameras)):
    camera = cameras[k]
    # get the csv files with the time information for the complete videos
    file = [x[2] for x in os.walk(join(data_folder, camera , 'CSV', 'complete'))]
    directory = join(data_folder, camera , 'CSV', 'complete', file[0][0])
    print(directory)
    synchro.play(directory, camera)
    # store the data in the variables
    index[camera] = synchro.data[camera]['index']
    relative_time[camera] = synchro.data[camera]['relative_time']
    absolute_time[camera] = synchro.data[camera]['abs_time']


../data_exp/70622/1/1/GOPRO/CSV/complete/outputGOPRO_abs_time.csv
../data_exp/70622/1/1/GOPRO2/CSV/complete/outputGOPRO2_abs_time.csv
../data_exp/70622/1/1/GOPRO3/CSV/complete/outputGOPRO3_abs_time.csv
../data_exp/70622/1/1/MICROSCOPE/CSV/complete/outputMICROSCOPE_abs_time.csv
../data_exp/70622/1/1/REALSENSE/CSV/complete/REALSENSE_abs_time.csv


## create and store absolute time data for the sensors (NEW RECORDER)

In [ ]:
#get sensor data from CSV
# NEW RECORDER: take one abs time ref, build new abs time vect based on relative time (frequency)

index_sensor = {}
relative_time_sensor = {}
absolute_time_sensor = {}
headers = {}
directory_sensor = {}
directory_sensor_new = {}

ref_abs_time_sensor = {}
new_abs_time_sensor = {}
new_abs_time_sensor_2 = {}
data_optitrack = {}
data_tps = {}


file = [x[2] for x in os.walk(join(data_folder, 'SENSORS'))]


for sensor in sensors:
    file = sensor + '.csv'
    print(file)
    directory_sensor[sensor] = join(data_folder, 'SENSORS', file)
    
    if os.path.exists(directory_sensor[sensor]):
        # for the EMG: at the end the recorder was not changed so the script writes the absolute time directly in the output CSV file
        # (based on the computer time)
        if sensor == "emg":
            synchro.play(directory_sensor[sensor], sensor)
            # store the data in variables
            index_sensor[sensor] = synchro.data[sensor]['index_global']
            relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
            absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
            
            f = open(directory_sensor[sensor])
            d_reader = csv.DictReader(f)
            #get fieldnames from DictReader object and store in list to have the name of the channels
            headers[sensor] = d_reader.fieldnames
            print(headers[sensor])
            
            # new absolute time is the same as the original one
            # use two variables to store data in a different format for the other sensors (here for emg the two are the same)
            new_abs_time_sensor[sensor] = absolute_time_sensor[sensor]
            new_abs_time_sensor_2[sensor] = absolute_time_sensor[sensor]
            
            # there is no new csv file so the "new" file is the existing one
            directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '.csv')
        
        
        if sensor == "optitrack":
            new_abs_time_sensor[sensor] = []
            
            f = open(directory_sensor[sensor])
            d_reader = csv.DictReader(f)
            #get fieldnames from DictReader object and store in list
            headers[sensor] = d_reader.fieldnames
            print(headers[sensor])
            
            # get the tools names from the csv
            tools = headers[sensor][3:]
            
            # store the data in variables
            synchro.play(directory_sensor[sensor], sensor)
            index_sensor[sensor] = synchro.data[sensor]['index']
            relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
            absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
            for tool in tools:
                data_optitrack[tool] = synchro.data[sensor][tool] #store data of each tool (x,y,z,quaternions)
            
            # create new csv file with the data AND the absolute time vector 
            opti_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
            f = open(opti_csv_new, 'w') 
            writer = csv.writer(f)
            # get the existing names of the columns
            header = headers[sensor]
            # add the new absolute time column
            header.append("absolute_time_new")
            writer.writerow(header)
            
            
            # Take one absolute time reference to build the new absolute time vector
            # the existing absolute time values are unprecise as they are coming from the python script when retrieveing the data (frequency varying a lot)
            # the relative timestamps are coming from motive = very reliable, frequency only changes when a data point is missed (still overload)
            # see report for explanations
            # one absolute time at the beginning is taken as reference but the relative timestamps are then used to avoid adding errors along the data
            # in the output csv, usually one or teo lines are useless (0values and negative relative time)
            # so we need to find the first line where we can get the absolute time ref
            T = True
            i = 0
            while T is True: # looks for the 1st good abs time value, then gets out of the loop

                if relative_time_sensor[sensor][i] > 10:
                    ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][i]
                    T = False
                i = i + 1
            
            # for all data points:
            for j in range(len(relative_time_sensor[sensor])-1):
                # if we are in the first useless lines, write the reference absolute time
                if j < i:
                    new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                # after: bluid the absolute time vector from the reference for each timestamps, based on the frequency of the relative timestamps
                else:
                    # take delta t of the relative time vector
                    diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                    # get corresponding value for the absolute time
                    new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff)
                    
                # put back the data of the original file (to have one complete CSV file)
                row = [index_sensor[sensor][j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                for tool in tools:
                    row.append(data_optitrack[tool][j])
                    
                # add a column with the new absolute time, correpsonding to each index
                row.append( new_abs_time_sensor[sensor][j])
                writer.writerow(row)
            f.close()
            
            # read again the data and store them in a new varible to have a panda serie instead of a list (each time linked with an index)
            file_new = sensor + '_new_abs_time.csv'
            new_abs_time_sensor_2[sensor] = pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new']
            
            # store the new csv name
            directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
            
            
        
        if sensor == "TPS_calibrated":
            new_abs_time_sensor[sensor] = []

            f = open(directory_sensor[sensor])
            d_reader = csv.DictReader(f)
            #get fieldnames from DictReader object and store in list
            headers[sensor] = d_reader.fieldnames
            print(headers[sensor])
            
            elements = headers[sensor][3:] #get the name of the columns (fingers)
            
            # store the data
            data = pd.read_csv(directory_sensor[sensor])
            abs_time_writing = data['abs_time_writing']
            relative_time_sensor[sensor] = data[' relative_time']
            absolute_time_sensor[sensor] = data[' absolute_time']
            for element in elements:
                data_tps[element] = data[element]
            
            # create new csv to add new absolute time
            tps_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
            f = open(tps_csv_new, 'w')            
            writer = csv.writer(f)
            
            # write the existing headers and add the column for the new absolute time
            header = headers[sensor] 
            header.append("absolute_time_new")
            writer.writerow(header)
            
            #for TPS: mostly same pb as for optitrack, writing the absolute time from the python script is not reliable
            # but the relative timestamps directly recorded are reliable
            # so only one reference absolute time is used and then the vector is built using the frequency of the relative timestamps
            # (avoid adding errors along the way, only the first absolute value is not very precise)            
            ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][0]
            for j in range(len(relative_time_sensor[sensor])-1):
                if j == 0:
                    # take the 1st absolute time for reference (no bad lines for tps)
                    new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                else:
                    # get the delta t from the relative timestamps to increment the new absolute timestamps, starting from the reference abs time
                    diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                    new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff )
                
                # get the original data to write in the ew file (have all data at the same place)
                row = [abs_time_writing [j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                for element in elements:
                    row.append(data_tps[element][j])
                
                # add the new absolute time vector (timestamps correponding to each data index)
                row.append(new_abs_time_sensor[sensor][j])
                writer.writerow(row)
                
        
            f.close()
            directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
            
            #read the written data to store them in a panda frame (with index for each timestamp)
            file_new = sensor + '_new_abs_time.csv'
            new_abs_time_sensor_2[sensor] = (pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new'])/1000 # CONVERSION UNITS 
            absolute_time_sensor[sensor] = absolute_time_sensor[sensor]/1000 # CONVERSION UNITS (to have same unit as for emg and optitrack)



## create and store absolute time data for the sensors (old RECORDER, for tests)

In [ ]:
"""#get sensor data from CSV

index_sensor = {}
relative_time_sensor = {}
absolute_time_sensor = {}
headers = {}
directory_sensor = {}
file = [x[2] for x in os.walk(join(data_folder, 'SENSORS'))]

for sensor in sensors:
    file = sensor + '.csv'
    print(file)
    
    directory_sensor[sensor] = join(data_folder, 'SENSORS', file)
    if os.path.exists(directory_sensor[sensor]):
        synchro.play(directory_sensor[sensor], sensor)
        index_sensor[sensor] = synchro.data[sensor]['index']
        relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
        absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
    
        f = open(directory_sensor[sensor])
        d_reader = csv.DictReader(f)
        #get fieldnames from DictReader object and store in list
        headers[sensor] = d_reader.fieldnames
        print(headers[sensor])"""

# PART 4: export segments

## Microscope as reference (NEW RECORDER)

### export ALL VIEWS FROM MICROSCOPE (by getting closest no of frame and rel time to the reference frame (start and stop frame of the segment on the microscope) using the absolute time, export the corresponding videos segments

In [ ]:
camera_ref = "MICROSCOPE"
cameras_2 = ['GOPRO', 'GOPRO2', 'MICROSCOPE', 'GOPRO3']


################## get the frames number corresponding to each segment(from microscope video) from the CSV file created manually ###########
directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_micro_frames.csv')
print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['start_frame']
stop_ref_frame_vector = DATA['stop_frame']
index_segment_vector = DATA['nb']


################################################ segmentation ##################################

# for all the defined segments
for o in range(len(start_ref_frame_vector)):
    #store the start and stop frame
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]
    
    # find the absolute time corresponding to the start and stop frames (of the camera used as reference: here microscope)
    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    
    
    pos_start = {}
    pos_stop = {}
    rel_time_start = {}
    rel_time_stop = {}
    abs_time_start = {}
    abs_time_stop = {}
    duration_clip = {}
    clip = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    
    # for all cameras except RS
    for camera in cameras_2:
        pos_start[camera] = []
        pos_stop[camera] = []
        rel_time_start[camera] = {}
        rel_time_stop[camera] = {}
        abs_time_start[camera] = {}
        abs_time_stop[camera] = {}
        duration_clip[camera] = {}
        
        # if it is not the reference camera (i.e not microscope):
        if not camera in camera_ref:
            # get index and absolute time of the frame with the absolute time closest to the start and stop absolute times, for the other cameras
            [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera], sensor)
            print(camera, "start index:       ", pos_start[camera], "abs time:   ", abs_time_start[camera])
            [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera], sensor)
            print(camera, "stop index:        ", pos_stop[camera], "abs time:   ", abs_time_stop[camera])
            
            #store the corresponding relative time to export the videos (videos have the relative time)
            rel_time_start[camera] = relative_time[camera][pos_start[camera]]
            rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]
            print(camera, "start index:       ", pos_start[camera], "rel time:   ", rel_time_start[camera])
            print(camera, "stop index:       ", pos_stop[camera], "rel time:   ", rel_time_stop[camera])
            print("----------------------")
        
        # for the reference camera, store the reference frames and time
        else:
            rel_time_start[camera] = relative_time[camera][start_ref_frame]
            rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]
            abs_time_start[camera] = start_ref_abs_time
            abs_time_stop[camera] = stop_ref_abs_time
            print(camera, "start index   ", start_ref_frame, "rel time:   ", rel_time_start[camera])
            print(camera, "stop index   ", stop_ref_frame, "rel time:   ", rel_time_stop[camera])
            print("----------------------")
            pos_start[camera] = start_ref_frame
            pos_stop[camera] = stop_ref_frame
            
            
        ################################################ export the corresponding individual videos ###############################################
                 
        #export videos from t1 to t2
        t1 = rel_time_start[camera]
        t2 = rel_time_stop[camera]
    
        # name of the exported videos:
        #name with no of frame
        #r1 = pos_start[camera]
        #r2 = pos_stop[camera]
        #ref = str(r1) + "_to_" + str(r2) + ".mp4"

        #name with segment nb
        r1 = index_segment_vector[o]
        ref = "segment_nb_" + str(r1) + ".mp4"
        
        # get the video file (complete videos)
        file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
        print(file[0][0])  
        target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
        file_name = join(data_folder, camera, 'complete', file[0][0])
        print(target_name)
        
        # condition: if the videos don't already exist: export them (to avoid losing a lot of time if we need to run this again but videos alreadz exist)
        # CAUTION: DOES NOT OVERWRITE VIDEOS WITH THIS CONDITION
        if not os.path.exists(target_name):
            ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)
        
        # save the independant views to re use them after
        clip[camera] = VideoFileClip(target_name)
        duration_clip[camera] = t2 - t1
            
    
    ############################################# export the 4 views together in 1 video##################################################
    ref2 = "segment_nb_" + str(r1) + "_all_views.mp4"
    output_name = join(path_to_data_folder, recording_session, subject, 'ALL_VIEWS', ref2)
    
    # condition: if the video doesn't already exist: export it
    # CAUTION: DOES NOT OVERWRITE VIDEOS WITH THIS CONDITION
    if not os.path.exists(output_name):
        # take smallest duration of the 4 videos (not exactly the same as not the same fps and exact time ref)
        smallest_duration = min(duration_clip.values())
        
        # create a black screen video if you want to have only 3 videos
        output_blank = join(path_to_data_folder, recording_session, subject, 'blank_video.mp4')
        size = (200, 100)
        duration = (smallest_duration)
        if not os.path.exists(output_blank):
            synchro.color_clip(size, duration, output_blank)
        clip4 = VideoFileClip(output_blank)

        # choose the clips you want to put together
        # if 3 clips:
        #clips = [[clip['MICROSCOPE'], clip4],
                 #[clip['GOPRO2'], clip['GOPRO']]]
        # if 4 clips:
        clips = [[clip['MICROSCOPE'], clip['GOPRO3']],
                 [clip['GOPRO2'], clip['GOPRO']]]
        
        # stacking clips
        final = clips_array(clips, rows_widths=[1000, 1000], cols_widths=[2000, 2000])

        # showing final clip
        #final.ipython_display(width = 480)

        # create video
        final.set_duration(smallest_duration).write_videofile(output_name, fps=30, remove_temp=True, audio = False)
        


### export the data segments from the microscope reference

In [ ]:
camera_ref = "MICROSCOPE"
cameras_2 = ['GOPRO', 'GOPRO2', 'MICROSCOPE', 'GOPRO3']

# after tests: saw that an offset was neede for the tps data to match the videos (amount determined by testing)
# the origin of this issue was not fixed (might come from the fact that the abs time is taken from the C++ script,
# while for the other sensors it is taken from the python script, but should be the absolute somputer time in both cases)
# define the offset in seconds and get corresponding number of data points using the approximate recording frequency
offset = 0.3 #[s]
offset_sensor = {}
frequency_sensor = {
    "TPS_calibrated" : 40, 
    "emg" : 0,
    "optitrack" : 0,
} 

################## get the frames number corresponding to each segment(from microscope video) from the CSV file created manually ###########
file = [x[2] for x in os.walk(join(data_folder, 'SEGMENTATION_CSV'))]
print(file[0][0])  
directory = join(data_folder ,'SEGMENTATION_CSV', file[0][0])
print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['start_frame']
stop_ref_frame_vector = DATA['stop_frame']
index_segment_vector = DATA['nb']

################################################ segmentation ##################################

# for all the defined segments    
for o in range(len(start_ref_frame_vector)):
    #get start and stop frame ref, and coresponding absolute time
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]

    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    
    pos_start_sensor = {}
    pos_stop_sensor = {}
    abs_time_start_sensor = {}
    abs_time_stop_sensor = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")  

    for sensor in sensors:
        # if we have the data for this sensor
        if os.path.exists(directory_sensor_new[sensor]):
            # can add a constant offset (here for tps only)
            offset_sensor[sensor] = math.floor((offset * frequency_sensor[sensor]))
            
            pos_start_sensor[sensor] = []
            pos_stop_sensor[sensor] = []
            abs_time_start_sensor[sensor] = {}
            abs_time_stop_sensor[sensor] = {}
            print(sensor)

            # find the index and absolute time corresponding to the closest absolute time ref of the start and stop timestamps
            [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, start_ref_abs_time, 0, sensor)
            print(sensor, "start index:       ", pos_start_sensor[sensor], "     abs time:   ", abs_time_start_sensor[sensor])
            [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, stop_ref_abs_time, 0, sensor)
            print(sensor, "stop index:        ", pos_stop_sensor[sensor], "      abs time:   ", abs_time_stop_sensor[sensor])
            
            # build a csv for each segment with the number of the segment
            r1 = index_segment_vector[o]
            ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
            csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)
            print(ref)
            f = open(csv_name, 'w') 
            writer = csv.writer(f)
            writer.writerow(headers[sensor])
            
            # create table with all the corresponding data for the segments (reading data from the original csv file)
            table = pd.read_csv(directory_sensor_new [sensor],skiprows=pos_start_sensor[sensor]-1 + offset_sensor[sensor] ,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+ 1 +offset_sensor[sensor]))
            table2 = table.to_numpy()
            
            # write in the new csv the data of interest 
            for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1 + offset_sensor[sensor]):
                row = table2[m]
                writer.writerow(row)

            f.close()        

# there can be some missing data if the sensors stopped before the end of the recording (usually the emg, sometimes all sensors)
# in this case we can sometimes not extract some segments and we have errors 
# these errors are not managed in this code

## Microscope as reference (old RECORDER) : used RS camera during the task, only 2 gopros for tests

In [ ]:
"""
camera_ref = "MICROSCOPE"
#get ref frames of microscope directly from csv file
file = [x[2] for x in os.walk(join(data_folder, 'SEGMENTATION_CSV'))]
print(file[0][0])  
directory = join(data_folder ,'SEGMENTATION_CSV', file[0][0])

print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['start_frame']
stop_ref_frame_vector = DATA['stop_frame']
index_segment_vector = DATA['nb']

    
for o in range(len(start_ref_frame_vector)):
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]

    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]

    pos_start = {}
    pos_stop = {}
    rel_time_start = {}
    rel_time_stop = {}
    abs_time_start = {}
    abs_time_stop = {}
    duration_clip = {}
    clip = {}
    
    pos_start_sensor = {}
    pos_stop_sensor = {}
    abs_time_start_sensor = {}
    abs_time_stop_sensor = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    
    for camera in cameras:
        pos_start[camera] = []
        pos_stop[camera] = []
        rel_time_start[camera] = {}
        rel_time_stop[camera] = {}
        abs_time_start[camera] = {}
        abs_time_stop[camera] = {}
        duration_clip[camera] = {}
        if not camera in camera_ref:
            [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera])
            print(camera, "start index:       ", pos_start[camera], "abs time:   ", abs_time_start[camera])
            [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera])
            print(camera, "stop index:        ", pos_stop[camera], "abs time:   ", abs_time_stop[camera])
            rel_time_start[camera] = relative_time[camera][pos_start[camera]]
            rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]
            print(camera, "start index:       ", pos_start[camera], "rel time:   ", rel_time_start[camera])
            print(camera, "stop index:       ", pos_stop[camera], "rel time:   ", rel_time_stop[camera])
            print("----------------------")
        else:
            rel_time_start[camera] = relative_time[camera][start_ref_frame]
            rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]
            abs_time_start[camera] = start_ref_abs_time
            abs_time_stop[camera] = stop_ref_abs_time
            print(camera, "start index   ", start_ref_frame, "rel time:   ", rel_time_start[camera])
            print(camera, "stop index   ", stop_ref_frame, "rel time:   ", rel_time_stop[camera])
            print("----------------------")
            pos_start[camera] = start_ref_frame
            pos_stop[camera] = stop_ref_frame
            
        #export video from t1 to t2 in seconds
        t1 = rel_time_start[camera]
        t2 = rel_time_stop[camera]
    
        #name with abs time ref
        #r1 = abs_time_start[camera]
        #r2 = abs_time_stop[camera]
        #ref = str(r1)[1:7] + "_to_" + str(r2)[1:7] + ".mp4"
    
        #name with no of frame
        r1 = pos_start[camera]
        r2 = pos_stop[camera]
        ref = str(r1) + "_to_" + str(r2) + ".mp4"

        #name with rel time
        #ref = str(t1)[1:7] + "_to_" + str(t2)[1:7] + ".mp4"
    
        #name with rsegment nb
        r1 = index_segment_vector[o]
        ref = "segment_nb_" + str(r1) + ".mp4"
        
        file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
        print(file[0][0])  
        target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
        file_name = join(data_folder, camera, 'complete', file[0][0])
        print(target_name)
        ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)
        
        clip[camera] = VideoFileClip(target_name)
        duration_clip[camera] = t2 - t1
        
    smallest_duration = min(duration_clip.values())
    output_blank = join(path_to_data_folder, recording_session, subject, 'blank_video.mp4')

    size = (200, 100)
    duration = (smallest_duration)
    print(duration)
    synchro.color_clip(size, duration, output_blank)
    clip4 = VideoFileClip(output_blank)
        
    # clips list
    clips = [[clip['MICROSCOPE'], clip['REALSENSE'].resize(1.4)],
             [clip['GOPRO2'], clip['GOPRO']]]
 
    # stacking clips
    final = clips_array(clips, rows_widths=[1000, 1000], cols_widths=[2000, 2000])
 
    # showing final clip
    #final.ipython_display(width = 480)
    ref2 = "segment_nb_" + str(r1) + "_all_views.mp4"
    output_name = join(path_to_data_folder, recording_session, subject, 'ALL_VIEWS', ref2)
    final.set_duration(smallest_duration).write_videofile(output_name, fps=30, remove_temp=True, audio = False)
      
    if os.path.exists(directory_sensor[sensor]):
        for sensor in sensors:
            pos_start_sensor[sensor] = []
            pos_stop_sensor[sensor] = []
            abs_time_start_sensor[sensor] = {}
            abs_time_stop_sensor[sensor] = {}

            [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(absolute_time_sensor[sensor]*1000, start_ref_abs_time, 0)
            print(sensor, "start index:       ", pos_start_sensor[sensor], "     abs time:   ", abs_time_start_sensor[sensor])
            [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(absolute_time_sensor[sensor]*1000, stop_ref_abs_time, 0)
            print(sensor, "stop index:        ", pos_stop_sensor[sensor], "      abs time:   ", abs_time_stop_sensor[sensor])

            r1 = index_segment_vector[o]
            ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
            csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)

            f = open(csv_name, 'w', newline='') 
            writer = csv.writer(f)
            writer.writerow(headers[sensor])
            table = pd.read_csv(directory_sensor[sensor],skiprows=pos_start_sensor[sensor]-1,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1))
            table2 = table.to_numpy()

            for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1):
                row = table2[m]
                writer.writerow(row)

            f.close()        

"""

## GOPRO2 as reference

### export ALL VIEWS FROM GOPRO2 (by getting closest no of frame and rel time to the reference frame (start and stop frame of the segment on the gopro2) using the absolute time, export the corresponding videos segments)

In [ ]:
# this allows to segment the videos and data using the video of the GOPRO2 as reference for the frame numbers 
#(if we wnt to segment based on the hand motions for example)

camera_ref = "GOPRO2"
cameras_2 = ['GOPRO', 'GOPRO2', 'GOPRO3','MICROSCOPE']


############################# get ref frames of gopro2 directly from csv file#############################################################
    
directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_gopro2_frames.csv')
print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['start_frame']
stop_ref_frame_vector = DATA['stop_frame']
index_segment_vector = DATA['nb']

##########################################################################################################################################
# from here: same code as for microcope (only need to differentiate the camera_ref and cameras_2 and the csv file)
# --> could choose only the first part and then use the same code (can be improved)
############################## segmentation ###############################################################################################

# for every segments
for o in range(len(start_ref_frame_vector)):
    # get start and stop frames
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]
    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    
    pos_start = {}
    pos_stop = {}
    rel_time_start = {}
    rel_time_stop = {}
    abs_time_start = {}
    abs_time_stop = {}
    duration_clip = {}
    clip = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    for camera in cameras_2:
        pos_start[camera] = []
        pos_stop[camera] = []
        rel_time_start[camera] = {}
        rel_time_stop[camera] = {}
        abs_time_start[camera] = {}
        abs_time_stop[camera] = {}
        duration_clip[camera] = {}
        # if it is not the reference camera (i.e not gopro2):
        if not camera in ["GOPRO2"]:
            # get index and absolute time of the frame with the absolute time closest to the start and stop absolute times, for the other cameras
            [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera], sensor)
            print(camera, "start index:       ", pos_start[camera], "abs time:   ", abs_time_start[camera])
            [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera], sensor)
            print(camera, "stop index:        ", pos_stop[camera], "abs time:   ", abs_time_stop[camera])
            
            # get corresponding relative time for video segmentation
            rel_time_start[camera] = relative_time[camera][pos_start[camera]]
            rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]
            print(camera, "start index:       ", pos_start[camera], "rel time:   ", rel_time_start[camera])
            print(camera, "stop index:       ", pos_stop[camera], "rel time:   ", rel_time_stop[camera])
            print("----------------------")
            
        # for the reference camera, store the reference frames and time
        else:
            rel_time_start[camera] = relative_time[camera][start_ref_frame]
            rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]
            abs_time_start[camera] = start_ref_abs_time
            abs_time_stop[camera] = stop_ref_abs_time
            print(camera, "start index   ", start_ref_frame, "rel time:   ", rel_time_start[camera])
            print(camera, "stop index   ", stop_ref_frame, "rel time:   ", rel_time_stop[camera])
            print("----------------------")
            pos_start[camera] = start_ref_frame
            pos_stop[camera] = stop_ref_frame
            
################################################ export the corresponding individual videos ###############################################
                 
        #export videos from t1 to t2
        t1 = rel_time_start[camera]
        t2 = rel_time_stop[camera]

        # name of the exported videos:
        #name with no of frame
        #r1 = pos_start[camera]
        #r2 = pos_stop[camera]
        #ref = str(r1) + "_to_" + str(r2) + ".mp4"

        #name with segment nb
        r1 = index_segment_vector[o]
        ref = "segment_nb_" + str(r1) + ".mp4"
        
        # get the video file (complete videos)
        file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
        print(file[0][0])  
        target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
        file_name = join(data_folder, camera, 'complete', file[0][0])
        print(target_name)
        
        # condition: if the videos don't already exist: export them (to avoid losing a lot of time if we need to run this again but videos alreadz exist)
        # CAUTION: DOES NOT OVERWRITE VIDEOS WITH THIS CONDITION
        if not os.path.exists(target_name):
            ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)
        
        # save the independant views to re use them after
        clip[camera] = VideoFileClip(target_name)
        duration_clip[camera] = t2 - t1
            
            
############################################# export the 4 views together in 1 video##################################################            
    ref2 = "segment_nb_" + str(r1) + "_all_views.mp4"
    output_name = join(path_to_data_folder, recording_session, subject, 'ALL_VIEWS', ref2)
    
    # condition: if the video doesn't already exist: export it
    # CAUTION: DOES NOT OVERWRITE VIDEOS WITH THIS CONDITION
    if not os.path.exists(output_name):
        # take smallest duration of the 4 videos (not exactly the same as not the same fps and exact time ref)
        smallest_duration = min(duration_clip.values())
        
        # create a black screen video if you want to have only 3 videos
        output_blank = join(path_to_data_folder, recording_session, subject, 'blank_video.mp4')
        size = (200, 100)
        duration = (smallest_duration)
        print(duration)
        if not os.path.exists(output_blank):
            synchro.color_clip(size, duration, output_blank)
        clip4 = VideoFileClip(output_blank)
        
        # choose the clips you want to put together
        # if realsense view:
        #clips = [[clip['MICROSCOPE'], clip['REALSENSE'].resize(1.4)],
                 #[clip['GOPRO2'], clip['GOPRO']]]

        # clips list with 4 views:
        clips = [[clip['MICROSCOPE'], clip['GOPRO3']],
                 [clip['GOPRO2'], clip['GOPRO']]]
        
        # stacking clips
        final = clips_array(clips, rows_widths=[1000, 1000], cols_widths=[2000, 2000])

        # showing final clip
        #final.ipython_display(width = 480)

        #create video
        final.set_duration(smallest_duration).write_videofile(output_name, fps=30, remove_temp=True, audio = False)
        


### export the data segments from the gopro2 reference

In [ ]:
camera_ref = "GOPRO2"
cameras_2 = ['GOPRO', 'GOPRO2', 'MICROSCOPE']

# after tests: saw that an offset was neede for the tps data to match the videos (amount determined by testing)
# the origin of this issue was not fixed (might come from the fact that the abs time is taken from the C++ script,
# while for the other sensors it is taken from the python script, but should be the absolute somputer time in both cases)
# define the offset in seconds and get corresponding number of data points using the approximate recording frequency
offset = 0.35 #[s]
offset_sensor = {}

frequency_sensor = {
    "TPS_calibrated" : 40,
    "emg" : 0,
    "optitrack" : 0,
} 


################## get the frames number corresponding to each segment(from microscope video) from the CSV file created manually ###########    
directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_gopro2_frames.csv')
print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['start_frame']
stop_ref_frame_vector = DATA['stop_frame']
index_segment_vector = DATA['nb']

################################################ segmentation ##################################

# for all the defined segments  
for o in range(len(start_ref_frame_vector)):
    # get the absolute time and frames for start and stop
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]
    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    
    pos_start_sensor = {}
    pos_stop_sensor = {}
    abs_time_start_sensor = {}
    abs_time_stop_sensor = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    
    for sensor in sensors:
        if os.path.exists(directory_sensor_new[sensor]):
            # add constant offset if needed
            offset_sensor[sensor] = math.floor((offset * frequency_sensor[sensor]))
            
            pos_start_sensor[sensor] = []
            pos_stop_sensor[sensor] = []
            abs_time_start_sensor[sensor] = {}
            abs_time_stop_sensor[sensor] = {}
            print(sensor)
            
            # find the index and absolute time corresponding to the closest absolute time ref of the start and stop timestamps
            [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, start_ref_abs_time, 0, sensor)
            print(sensor, "start index:       ", pos_start_sensor[sensor], "     abs time:   ", abs_time_start_sensor[sensor])
            [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, stop_ref_abs_time, 0, sensor)
            print(sensor, "stop index:        ", pos_stop_sensor[sensor], "      abs time:   ", abs_time_stop_sensor[sensor])

            # build a csv for each segment with the number of the segment
            r1 = index_segment_vector[o]
            ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
            csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)
            f = open(csv_name, 'w') 
            writer = csv.writer(f)
            writer.writerow(headers[sensor])
            
            # create table with all the corresponding data for the segments (reading data from the original csv file)
            table = pd.read_csv(directory_sensor_new [sensor],skiprows=pos_start_sensor[sensor]-1 + offset_sensor[sensor] ,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+ 1 +offset_sensor[sensor]))
            table2 = table.to_numpy()
            
            # write in the new csv the data of interest 
            for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1 + offset_sensor[sensor]):
                row = table2[m]
                writer.writerow(row)

            f.close()   

# there can be some missing data if the sensors stopped before the end of the recording (usually the emg, sometimes all sensors)
# in this case we can sometimes not extract some segments and we have errors 
# these errors are not managed in this code

# PART 5: Complete working script for segmentation (when you have to do all the steps after having the videos converted, to avoid having to run all the cells for each recording)

## complete code for exporting videos and segments from microscope

In [ ]:
########################################################### MANUAL inputs################################
recording_session = '80622'
subjects = ['5/4']
fps_gopro = 59.940   
fps_rs = 30
fps_micro = 30       

sensors = ['optitrack', 'TPS_calibrated','emg']

path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                


cameras = ['GOPRO', 'GOPRO2', 'GOPRO3', 'MICROSCOPE', 'REALSENSE']
folder = ['complete','segments']

folder_creation_all = ['CSV', 'SEGMENTATION', 'complete', 'segments']
folder_creation_rs = ['PNG', 'BAG']
folder_creation_global = ['ALL_VIEWS']
        
fps = {
    "GOPRO": fps_gopro,
    "GOPRO2": fps_gopro,
    "GOPRO3": fps_gopro,
    "MICROSCOPE": fps_micro,
    "REALSENSE": fps_rs
}

################################################# READ SYNCHRO FRAMES ##########################################3
for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject)
    file = 'synchro_frame.csv'
    directory = join(data_folder, file)

    data = pd.read_csv(directory)
    rs_frame = int(data['rs_frame'])
    micro_frame = int(data['micro_frame'])
    gopro_frame = int(data['gopro_frame'])
    gopro2_frame = int(data['gopro2_frame'])
    gopro3_frame = int(data['gopro3_frame'])


    frame = {
        "REALSENSE": rs_frame,
        "GOPRO": gopro_frame,
        "GOPRO2": gopro2_frame,
        "GOPRO3": gopro3_frame,
        "MICROSCOPE": micro_frame,

    }

########################################################## GET NUMBER FO FRAMES #################################################
    camera = 'REALSENSE'
    image_folder=join(data_folder, camera, 'PNG')
    image_files = [os.path.join(image_folder,img) for img in os.listdir(image_folder) if img.endswith(".png")]

    # get nb of frames of the rs camera
    rs_nb_frame = len(image_files)

    # get absolute time vector of rs from png files
    abs_time_vector = {}
    abs_time_vector[camera] = [name[-32:-4] for name in natsorted(image_files)]

    rs_abs_time = float(abs_time_vector['REALSENSE'][rs_frame]) #abs time vector from png names

    # get the nub of frames for each segments and the total videos for the gopro and microscope

    nb_frames = {}
    sum_frame = {}

    for k in range(len(cameras)):   
        camera = cameras[k]
        sum_frame[camera] = {}
        for j in range(len(folder)):  
            files = [x[2] for x in os.walk(join(data_folder, camera,folder[j]))]   
            tmp = 0
            if camera == 'REALSENSE':
                        nb_frames[camera] = rs_nb_frame
            else:
                if len(files) != 0:                                                    
                    for i in range(len(files[0])):                                   
                        file = files[0][i]                                            
                        if os.path.splitext(file)[1] == '.MP4' or os.path.splitext(file)[1] == '.mp4':   
                            cap = cv2.VideoCapture(join(data_folder, camera, folder[j], file))
                            nb_frames[file] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                     

                            if folder[j] == 'segments':                                
                                tmp = tmp + nb_frames[file]
                                sum_frame[camera] = tmp

####################################################### CREATE CSV WITH ABS TIME FOR THE CAMERAS ######################################
    csv_names = []

    for k in range(len(cameras)):
        camera = cameras[k]

        for j in range(len(folder)):
            files = [x[2] for x in os.walk(join(data_folder, camera ,folder[j]))]
            print(files)  

            if camera == 'REALSENSE':

                rs_csv = join(data_folder,camera, 'CSV', 'complete', 'REALSENSE_abs_time.csv')
                f = open(rs_csv, 'w')
                writer = csv.writer(f)
                header = ['index','frequency','relative_time','abs_time']
                writer.writerow(header)
                rel_time = 0

                for m in range((nb_frames[camera])):                            
                    abs_time = abs_time_vector['REALSENSE'][m]
                    row = [m,fps[camera],rel_time,float(abs_time)]               
                    writer.writerow(row)
                    rel_time = rel_time + 1/fps[camera]                        
                f.close()        

            else:
                for i in range(len(files[0])):
                    file = files[0][i] 
                    if os.path.splitext(file)[1] == '.mp4' or os.path.splitext(file)[1] == '.MP4':

                        csv_names.append(join(data_folder,camera, 'CSV',folder[j], str(file)[0:-4] + '_abs_time.csv'))  
                        f = open(csv_names[-1], 'w') 
                        writer = csv.writer(f)
                        header = ['index','frequency','relative_time','abs_time']
                        writer.writerow(header)
                        rel_time = 0
                        abs_time = rs_abs_time                                      

                        for m in range((nb_frames[file])):                            
                            if folder[j] == 'complete' and m >= frame[camera]:        
                                row = [m,fps[camera],rel_time,abs_time]               
                                abs_time = abs_time + 1/fps[camera]*1000
                            else:
                                row = [m, fps[camera], rel_time]                     

                            writer.writerow(row)
                            rel_time = rel_time + 1/fps[camera]                      
                        f.close()

    index = {}
    relative_time = {}
    absolute_time = {}

    for k in range(len(cameras)):
        camera = cameras[k]
        file = [x[2] for x in os.walk(join(data_folder, camera , 'CSV', 'complete'))]
        print(file[0][0])  

        directory = join(data_folder, camera , 'CSV', 'complete', file[0][0])
        print(directory)
        synchro.play(directory, camera)
        index[camera] = synchro.data[camera]['index']
        relative_time[camera] = synchro.data[camera]['relative_time']
        absolute_time[camera] = synchro.data[camera]['abs_time']

########################################################## CREATE CSV FILE WITH ABS TIME FOR THE SENSORS #####################################################

    index_sensor = {}
    relative_time_sensor = {}
    absolute_time_sensor = {}
    headers = {}
    directory_sensor = {}
    directory_sensor_new = {}

    ref_abs_time_sensor = {}
    new_abs_time_sensor = {}
    new_abs_time_sensor_2 = {}
    data_optitrack = {}
    data_tps = {}


    file = [x[2] for x in os.walk(join(data_folder, 'SENSORS'))]

    for sensor in sensors:
        file = sensor + '.csv'

        print(file)

        directory_sensor[sensor] = join(data_folder, 'SENSORS', file)
        if os.path.exists(directory_sensor[sensor]):

            if sensor == "emg": 
                synchro.play(directory_sensor[sensor], sensor)
                index_sensor[sensor] = synchro.data[sensor]['index_global']
                relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
                absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames
                new_abs_time_sensor[sensor] = absolute_time_sensor[sensor]
                new_abs_time_sensor_2[sensor] = absolute_time_sensor[sensor]

                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '.csv')


            if sensor == "optitrack":
                new_abs_time_sensor[sensor] = []

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames

                tools = headers[sensor][3:]

                synchro.play(directory_sensor[sensor], sensor)
                index_sensor[sensor] = synchro.data[sensor]['index']
                relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
                absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
                for tool in tools:
                    data_optitrack[tool] = synchro.data[sensor][tool]

                opti_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
                f = open(opti_csv_new, 'w') 
                writer = csv.writer(f)
                header = headers[sensor]
                header.append("absolute_time_new")
                writer.writerow(header)
                T = True
                i = 0
                
                while T is True:
                    if relative_time_sensor[sensor][i] > 10:
                        ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][i]
                        T = False
                    i = i + 1

                for j in range(len(relative_time_sensor[sensor])-1):
                    if j < i:
                        new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                    else:
                        diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                        new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff)
                    row = [index_sensor[sensor][j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                    for tool in tools:
                        row.append(data_optitrack[tool][j])
                    row.append( new_abs_time_sensor[sensor][j])
                    writer.writerow(row)

                f.close()
                file_new = sensor + '_new_abs_time.csv'
                new_abs_time_sensor_2[sensor] = pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new']

                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')



            if sensor == "TPS_calibrated":
                new_abs_time_sensor[sensor] = []

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames
                elements = headers[sensor][3:]
                data = pd.read_csv(directory_sensor[sensor])

                abs_time_writing = data['abs_time_writing']
                relative_time_sensor[sensor] = data[' relative_time']
                absolute_time_sensor[sensor] = data[' absolute_time']
                for element in elements:
                    data_tps[element] = data[element]

                tps_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
                f = open(tps_csv_new, 'w')
                writer = csv.writer(f)
                header = headers[sensor] 
                header.append("absolute_time_new")
                writer.writerow(header)
                ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][0]

                for j in range(len(relative_time_sensor[sensor])-1):
                    if j == 0:
                        new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                    else:
                        diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                        new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff )
                    row = [abs_time_writing [j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                    for element in elements:
                        row.append(data_tps[element][j])

                    row.append(new_abs_time_sensor[sensor][j])
                    writer.writerow(row)


                f.close()
                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')


                file_new = sensor + '_new_abs_time.csv'
                new_abs_time_sensor_2[sensor] = (pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new'])/1000
                absolute_time_sensor[sensor] = absolute_time_sensor[sensor]/1000



##################################################### EXPORT VIDEOS #############################################################################333333

    camera_ref = "MICROSCOPE"
    cameras_2 = ['GOPRO', 'GOPRO2', 'MICROSCOPE', 'GOPRO3']

    directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_micro_frames.csv')
    print(directory)
    DATA = pd.read_csv(directory)
    start_ref_frame_vector = DATA['start_frame']
    stop_ref_frame_vector = DATA['stop_frame']
    index_segment_vector = DATA['nb']


    for o in range(len(start_ref_frame_vector)):
        start_ref_frame = start_ref_frame_vector[o]
        print(start_ref_frame)
        stop_ref_frame = stop_ref_frame_vector[o]

        #camera = camera_ref
        start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
        stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
        #
        pos_start = {}
        pos_stop = {}
        rel_time_start = {}
        rel_time_stop = {}
        abs_time_start = {}
        abs_time_stop = {}
        duration_clip = {}
        clip = {}

        print("start reference abs time:  ", start_ref_abs_time)  
        print("stop reference abs time:  ", stop_ref_abs_time) 
        print("----------------------")
        for camera in cameras_2:
            pos_start[camera] = []
            pos_stop[camera] = []
            rel_time_start[camera] = {}
            rel_time_stop[camera] = {}
            abs_time_start[camera] = {}
            abs_time_stop[camera] = {}
            duration_clip[camera] = {}
            if not camera in camera_ref:
                [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera], sensor)
                print(camera, "start index:       ", pos_start[camera], "abs time:   ", abs_time_start[camera])
                [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera], sensor)
                print(camera, "stop index:        ", pos_stop[camera], "abs time:   ", abs_time_stop[camera])
                rel_time_start[camera] = relative_time[camera][pos_start[camera]]
                rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]
                print(camera, "start index:       ", pos_start[camera], "rel time:   ", rel_time_start[camera])
                print(camera, "stop index:       ", pos_stop[camera], "rel time:   ", rel_time_stop[camera])
                print("----------------------")
            else:
                rel_time_start[camera] = relative_time[camera][start_ref_frame]
                rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]
                abs_time_start[camera] = start_ref_abs_time
                abs_time_stop[camera] = stop_ref_abs_time
                print(camera, "start index   ", start_ref_frame, "rel time:   ", rel_time_start[camera])
                print(camera, "stop index   ", stop_ref_frame, "rel time:   ", rel_time_stop[camera])
                print("----------------------")
                pos_start[camera] = start_ref_frame
                pos_stop[camera] = stop_ref_frame

            #export video from t1 to t2 in secondS
            t1 = rel_time_start[camera]
            t2 = rel_time_stop[camera]

            #name with no of frame
            #r1 = pos_start[camera]
            #r2 = pos_stop[camera]
            #ref = str(r1) + "_to_" + str(r2) + ".mp4"

            #name with rsegment nb
            r1 = index_segment_vector[o]
            ref = "segment_nb_" + str(r1) + ".mp4"

            file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
            print(file[0][0])  
            target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
            file_name = join(data_folder, camera, 'complete', file[0][0])
            print(target_name)
            if not os.path.exists(target_name):
                ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)

            clip[camera] = VideoFileClip(target_name)
            duration_clip[camera] = t2 - t1

        ref2 = "segment_nb_" + str(r1) + "_all_views.mp4"

        output_name = join(path_to_data_folder, recording_session, subject, 'ALL_VIEWS', ref2)
        if not os.path.exists(output_name):
            smallest_duration = min(duration_clip.values())
            
            output_blank = join(path_to_data_folder, recording_session, subject, 'blank_video.mp4')
            size = (200, 100)
            duration = (smallest_duration)
            print(duration)
            if not os.path.exists(output_blank):
                synchro.color_clip(size, duration, output_blank)
            clip4 = VideoFileClip(output_blank)

            # clips list
            #clips = [[clip['MICROSCOPE'], clip['REALSENSE'].resize(1.4)],
                     #[clip['GOPRO2'], clip['GOPRO']]]

            # clips list wo RS
            #clips = [[clip['MICROSCOPE'], clip4],
                     #[clip['GOPRO2'], clip['GOPRO']]]
            clips = [[clip['MICROSCOPE'], clip['GOPRO3']],
                     [clip['GOPRO2'], clip['GOPRO']]]

            # stacking clips
            final = clips_array(clips, rows_widths=[1000, 1000], cols_widths=[2000, 2000])

            # showing final clip
            #final.ipython_display(width = 480)

            final.set_duration(smallest_duration).write_videofile(output_name, fps=30, remove_temp=True, audio = False)

################################################################## EXPORT DATA #################################################################################3            
    camera_ref = "MICROSCOPE"
    #sensors = ["optitrack", "TPS_calibrated"]
    cameras_2 = ['GOPRO', 'GOPRO2', 'MICROSCOPE', 'GOPRO3']
    offset = 0.3 #[s]
    offset_sensor = {}

    frequency_sensor = {
        "TPS_calibrated" : 40,
        "emg" : 0,
        "optitrack" : 0,
    } #approx
    import math

    #get ref fram of microscope directly from csv file
    file = [x[2] for x in os.walk(join(data_folder, 'SEGMENTATION_CSV'))]
    print(file[0][0])  

    directory = join(data_folder ,'SEGMENTATION_CSV', file[0][0])
    print(directory)
    DATA = pd.read_csv(directory)
    start_ref_frame_vector = DATA['start_frame']
    stop_ref_frame_vector = DATA['stop_frame']
    index_segment_vector = DATA['nb']


    for o in range(len(start_ref_frame_vector)):
        start_ref_frame = start_ref_frame_vector[o]
        print(start_ref_frame)
        stop_ref_frame = stop_ref_frame_vector[o]

        start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
        stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
        
        pos_start_sensor = {}
        pos_stop_sensor = {}
        abs_time_start_sensor = {}
        abs_time_stop_sensor = {}

        print("start reference abs time:  ", start_ref_abs_time)  
        print("stop reference abs time:  ", stop_ref_abs_time) 
        print("----------------------")

        for sensor in sensors:
            if os.path.exists(directory_sensor_new[sensor]):

                offset_sensor[sensor] = math.floor((offset * frequency_sensor[sensor]))

                pos_start_sensor[sensor] = []
                pos_stop_sensor[sensor] = []
                abs_time_start_sensor[sensor] = {}
                abs_time_stop_sensor[sensor] = {}
                print(sensor)

                [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, start_ref_abs_time, 0, sensor)
                print(sensor, "start index:       ", pos_start_sensor[sensor], "     abs time:   ", abs_time_start_sensor[sensor])
                [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, stop_ref_abs_time, 0, sensor)
                print(sensor, "stop index:        ", pos_stop_sensor[sensor], "      abs time:   ", abs_time_stop_sensor[sensor])

                r1 = index_segment_vector[o]
                ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
                csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)

                f = open(csv_name, 'w') 
                writer = csv.writer(f)
                writer.writerow(headers[sensor])
                
                table = pd.read_csv(directory_sensor_new [sensor],skiprows=pos_start_sensor[sensor]-1 + offset_sensor[sensor] ,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+ 1 +offset_sensor[sensor]))
                table2 = table.to_numpy()

                for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1 + offset_sensor[sensor]):
                    row = table2[m]
                    writer.writerow(row)

                f.close()        



## Complete script to export synchro segments from gopro2

## export all

In [ ]:

############################################## manual entries: to be adapted ###############################################
recording_session = '70622'
subjects = ['1/1'] # can put several subjets (recordings) for the loop
fps_gopro = 59.940   
fps_rs = 30
fps_micro = 30       

sensors = ['optitrack', 'emg', 'TPS_calibrated']
path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                 


cameras = ['GOPRO', 'GOPRO2', 'GOPRO3', 'MICROSCOPE', 'REALSENSE']
folder = ['complete','segments']

folder_creation_all = ['CSV', 'SEGMENTATION', 'complete', 'segments']
folder_creation_rs = ['PNG', 'BAG']
folder_creation_global = ['ALL_VIEWS']

        
fps = {
    "GOPRO": fps_gopro,
    "GOPRO2": fps_gopro,
    "GOPRO3": fps_gopro,
    "MICROSCOPE": fps_micro,
    "REALSENSE": fps_rs
}

############################################## READ SYNCHRO FRAMES #################################################################
for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject)

    file = 'synchro_frame.csv'
    directory = join(data_folder, file)

    data = pd.read_csv(directory)
    rs_frame = int(data['rs_frame'])
    micro_frame = int(data['micro_frame'])
    gopro_frame = int(data['gopro_frame'])
    gopro2_frame = int(data['gopro2_frame'])
    gopro3_frame = int(data['gopro3_frame'])

    frame = {
        "REALSENSE": rs_frame,
        "GOPRO": gopro_frame,
        "GOPRO2": gopro2_frame,
        "GOPRO3": gopro3_frame,
        "MICROSCOPE": micro_frame,

    }


############################################## GET ABS TIME FOR RS CAMERA FROM PNG NAMES ##################################
    camera = 'REALSENSE'
    image_folder=join(data_folder, camera, 'PNG')
    image_files = [os.path.join(image_folder,img) for img in os.listdir(image_folder) if img.endswith(".png")]

    # get nb of frames of the rs camera
    rs_nb_frame = len(image_files)

    # get absolute time vector of rs from png files
    abs_time_vector = {}
    abs_time_vector[camera] = [name[-32:-4] for name in natsorted(image_files)]

    rs_abs_time = float(abs_time_vector['REALSENSE'][rs_frame]) #abs time vector from png names


############################################# GET THE NUMBER OF FRAMES ####################################################

    nb_frames = {}
    sum_frame = {}
    for k in range(len(cameras)):    
        camera = cameras[k]
        sum_frame[camera] = {}
        for j in range(len(folder)):  
            files = [x[2] for x in os.walk(join(data_folder, camera,folder[j]))]   
            tmp = 0
            if camera == 'REALSENSE':
                        nb_frames[camera] = rs_nb_frame
            else:
                if len(files) != 0:                                                    
                    for i in range(len(files[0])):                                   
                        file = files[0][i]                                             

                        if os.path.splitext(file)[1] == '.MP4' or os.path.splitext(file)[1] == '.mp4':   
                            cap = cv2.VideoCapture(join(data_folder, camera, folder[j], file))
                            nb_frames[file] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                    

                            if folder[j] == 'segments':                                
                                tmp = tmp + nb_frames[file]
                                sum_frame[camera] = tmp

                                
#################################### CREATE CSV FILES WITH THE ABSOLUTE TIME REFERENCES for the cameras #######################################                                
    csv_names = []

    for k in range(len(cameras)):
        camera = cameras[k]

        for j in range(len(folder)):
            files = [x[2] for x in os.walk(join(data_folder, camera ,folder[j]))]
            print(files)  

            if camera == 'REALSENSE':

                rs_csv = join(data_folder,camera, 'CSV', 'complete', 'REALSENSE_abs_time.csv')
                f = open(rs_csv, 'w') 
                writer = csv.writer(f)
                header = ['index','frequency','relative_time','abs_time']
                writer.writerow(header)
                rel_time = 0
                
                for m in range((nb_frames[camera])):                            
                    abs_time = abs_time_vector['REALSENSE'][m]
                    row = [m,fps[camera],rel_time,float(abs_time)]               
                    writer.writerow(row)
                    rel_time = rel_time + 1/fps[camera]                      

                f.close()        

            else:
                for i in range(len(files[0])):
                    file = files[0][i] 
                    if os.path.splitext(file)[1] == '.mp4' or os.path.splitext(file)[1] == '.MP4':

                        csv_names.append(join(data_folder,camera, 'CSV',folder[j], str(file)[0:-4] + '_abs_time.csv'))  
                        f = open(csv_names[-1], 'w') 
                        writer = csv.writer(f)
                        header = ['index','frequency','relative_time','abs_time']

                        writer.writerow(header)
                        rel_time = 0
                        abs_time = rs_abs_time                                      

                        for m in range((nb_frames[file])):                            
                            if folder[j] == 'complete' and m >= frame[camera]:        
                                row = [m,fps[camera],rel_time,abs_time]               
                                abs_time = abs_time + 1/fps[camera]*1000
                            else:
                                row = [m, fps[camera], rel_time]        
                                
                            writer.writerow(row)
                            rel_time = rel_time + 1/fps[camera]                      

                        f.close()


    index = {}
    relative_time = {}
    absolute_time = {}

    for k in range(len(cameras)):
        camera = cameras[k]
        file = [x[2] for x in os.walk(join(data_folder, camera , 'CSV', 'complete'))]
        print(file[0][0])  

        directory = join(data_folder, camera , 'CSV', 'complete', file[0][0])
        print(directory)
        synchro.play(directory, camera)
        index[camera] = synchro.data[camera]['index']
        relative_time[camera] = synchro.data[camera]['relative_time']
        absolute_time[camera] = synchro.data[camera]['abs_time']

    
#################################### CREATE CSV FILES WITH THE ABSOLUTE TIME REFERENCES for the sensors #######################################                                

    index_sensor = {}
    relative_time_sensor = {}
    absolute_time_sensor = {}
    headers = {}
    directory_sensor = {}
    directory_sensor_new = {}

    ref_abs_time_sensor = {}
    new_abs_time_sensor = {}
    new_abs_time_sensor_2 = {}
    data_optitrack = {}
    data_tps = {}

    file = [x[2] for x in os.walk(join(data_folder, 'SENSORS'))]

    for sensor in sensors:
        file = sensor + '.csv'

        print(file)

        directory_sensor[sensor] = join(data_folder, 'SENSORS', file)
        if os.path.exists(directory_sensor[sensor]):

            if sensor == "emg": 
                synchro.play(directory_sensor[sensor], sensor)
                index_sensor[sensor] = synchro.data[sensor]['index_global']
                relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
                absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames
                new_abs_time_sensor[sensor] = absolute_time_sensor[sensor]
                new_abs_time_sensor_2[sensor] = absolute_time_sensor[sensor]

                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '.csv')

            if sensor == "optitrack":
                new_abs_time_sensor[sensor] = []

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames
                tools = headers[sensor][3:]

                synchro.play(directory_sensor[sensor], sensor)
                index_sensor[sensor] = synchro.data[sensor]['index']
                relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
                absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
                for tool in tools:
                    data_optitrack[tool] = synchro.data[sensor][tool]

                opti_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
                f = open(opti_csv_new, 'w')
                writer = csv.writer(f)
                header = headers[sensor]
                header.append("absolute_time_new")
                # write a row to the csv file
                writer.writerow(header)
                T = True
                i = 0
                
                while T is True:
                    if relative_time_sensor[sensor][i] > 10:
                        ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][i]
                        T = False
                    i = i + 1
                    
                for j in range(len(relative_time_sensor[sensor])-1):
                    if j < i:
                        new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                    else:
                        diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                        new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff)
                    row = [index_sensor[sensor][j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                    for tool in tools:
                        row.append(data_optitrack[tool][j])
                    row.append( new_abs_time_sensor[sensor][j])
                    writer.writerow(row)

                f.close()
                file_new = sensor + '_new_abs_time.csv'
                new_abs_time_sensor_2[sensor] = pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new']
                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')


            if sensor == "TPS_calibrated":
                new_abs_time_sensor[sensor] = []

                f = open(directory_sensor[sensor])
                d_reader = csv.DictReader(f)
                headers[sensor] = d_reader.fieldnames
                elements = headers[sensor][3:]

                data = pd.read_csv(directory_sensor[sensor])
                abs_time_writing = data['abs_time_writing']
                relative_time_sensor[sensor] = data[' relative_time']
                absolute_time_sensor[sensor] = data[' absolute_time']
                for element in elements:
                    data_tps[element] = data[element]

                tps_csv_new = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')
                f = open(tps_csv_new, 'w')
                writer = csv.writer(f)
                header = headers[sensor] 
                header.append("absolute_time_new")
                writer.writerow(header)

                ref_abs_time_sensor[sensor] = absolute_time_sensor[sensor][0]

                for j in range(len(relative_time_sensor[sensor])-1):
                    if j == 0:
                        new_abs_time_sensor[sensor].append(ref_abs_time_sensor[sensor])
                    else:
                        diff = (relative_time_sensor[sensor][j] - relative_time_sensor[sensor][j-1])
                        new_abs_time_sensor[sensor].append(new_abs_time_sensor[sensor][j-1] + diff )
                    row = [abs_time_writing [j], relative_time_sensor[sensor][j], absolute_time_sensor[sensor][j]]
                    for element in elements:
                        row.append(data_tps[element][j])

                    row.append(new_abs_time_sensor[sensor][j])
                    writer.writerow(row)


                f.close()
                directory_sensor_new [ sensor] = join(data_folder, 'SENSORS', sensor + '_new_abs_time.csv')


                file_new = sensor + '_new_abs_time.csv'
                new_abs_time_sensor_2[sensor] = (pd.read_csv(join(data_folder, 'SENSORS', file_new))['absolute_time_new'])/1000
                absolute_time_sensor[sensor] = absolute_time_sensor[sensor]/1000



############################################ EXPORT VIDEO SEGMENTS ###################################################################

    camera_ref = "GOPRO2"
    cameras_2 = ['GOPRO', 'GOPRO2', 'GOPRO3','MICROSCOPE']

    directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_gopro2_frames.csv')
    print(directory)

    DATA = pd.read_csv(directory)
    start_ref_frame_vector = DATA['start_frame']
    stop_ref_frame_vector = DATA['stop_frame']
    index_segment_vector = DATA['nb']


    for o in range(len(start_ref_frame_vector)):
        start_ref_frame = start_ref_frame_vector[o]
        stop_ref_frame = stop_ref_frame_vector[o]
        start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
        stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
        #
        pos_start = {}
        pos_stop = {}
        rel_time_start = {}
        rel_time_stop = {}
        abs_time_start = {}
        abs_time_stop = {}
        duration_clip = {}
        clip = {}


        for camera in cameras_2:
            pos_start[camera] = []
            pos_stop[camera] = []
            rel_time_start[camera] = {}
            rel_time_stop[camera] = {}
            abs_time_start[camera] = {}
            abs_time_stop[camera] = {}
            duration_clip[camera] = {}
            if not camera in camera_ref:
                [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera], sensor)
                [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera], sensor)
                rel_time_start[camera] = relative_time[camera][pos_start[camera]]
                rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]

            else:
                rel_time_start[camera] = relative_time[camera][start_ref_frame]
                rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]

                abs_time_start[camera] = start_ref_abs_time
                abs_time_stop[camera] = stop_ref_abs_time
    
                pos_start[camera] = start_ref_frame
                pos_stop[camera] = stop_ref_frame

            
            #export video from t1 to t2 in seconds
            t1 = rel_time_start[camera]
            t2 = rel_time_stop[camera]

            #name with no of frame
            #r1 = pos_start[camera]
            #r2 = pos_stop[camera]
            #ref = str(r1) + "_to_" + str(r2) + ".mp4"

            #name with rsegment nb
            r1 = index_segment_vector[o]
            ref = "segment_nb_" + str(r1) + ".mp4"

            file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
            print(file[0][0])  
            target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
            file_name = join(data_folder, camera, 'complete', file[0][0])
            #print(target_name)
            if not os.path.exists(target_name):
                ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)

            clip[camera] = VideoFileClip(target_name)
            duration_clip[camera] = t2 - t1

        ref2 = "segment_nb_" + str(r1) + "_all_views.mp4"

        output_name = join(path_to_data_folder, recording_session, subject, 'ALL_VIEWS', ref2)
        if not os.path.exists(output_name):

            smallest_duration = min(duration_clip.values())


            output_blank = join(path_to_data_folder, recording_session, subject, 'blank_video.mp4')

            size = (200, 100)
            duration = (smallest_duration)

            if not os.path.exists(output_blank):
                synchro.color_clip(size, duration, output_blank)
            clip4 = VideoFileClip(output_blank)


            clips = [[clip['MICROSCOPE'], clip['GOPRO3']],
                     [clip['GOPRO2'], clip['GOPRO']]]

            # stacking clips
            final = clips_array(clips, rows_widths=[1000, 1000], cols_widths=[2000, 2000])

            # showing final clip
            #final.ipython_display(width = 480)

            final.set_duration(smallest_duration).write_videofile(output_name, fps=30, remove_temp=True, audio = False)


############################################### EXPORT DATA SEGMENTS #####################################################################
    offset = 0.35 #[s]
    offset_sensor = {}

    frequency_sensor = {
        "TPS_calibrated" : 40,
        "emg" : 0,
        "optitrack" : 0,
    }

    directory = join(data_folder, 'SEGMENTATION_CSV', 'segmentation_gopro2_frames.csv')
    print(directory)

    DATA = pd.read_csv(directory)
    start_ref_frame_vector = DATA['start_frame']
    stop_ref_frame_vector = DATA['stop_frame']
    index_segment_vector = DATA['nb']


    for o in range(len(start_ref_frame_vector)):
        start_ref_frame = start_ref_frame_vector[o]
        stop_ref_frame = stop_ref_frame_vector[o]
        start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
        stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]

        pos_start_sensor = {}
        pos_stop_sensor = {}
        abs_time_start_sensor = {}
        abs_time_stop_sensor = {}


        for sensor in sensors:
            if os.path.exists(directory_sensor_new[sensor]):

                offset_sensor[sensor] = math.floor((offset * frequency_sensor[sensor]))

                pos_start_sensor[sensor] = []
                pos_stop_sensor[sensor] = []
                abs_time_start_sensor[sensor] = {}
                abs_time_stop_sensor[sensor] = {}

                [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, start_ref_abs_time, 0, sensor)
                [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(new_abs_time_sensor_2[sensor]*1000, stop_ref_abs_time, 0, sensor)

                r1 = index_segment_vector[o]
                ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
                csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)
                f = open(csv_name, 'w') 
                writer = csv.writer(f)
                writer.writerow(headers[sensor])
                
                table = pd.read_csv(directory_sensor_new [sensor],skiprows=pos_start_sensor[sensor]-1 + offset_sensor[sensor] ,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+ 1 +offset_sensor[sensor]))
                table2 = table.to_numpy()

                for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1 + offset_sensor[sensor]):
                    row = table2[m]
                    writer.writerow(row)

                f.close()        



# PLOTS

## Plot the synchronization movements data to check good synchronization between exported videos and data

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

############################################################# choose what to plot ##################################################
recording_session = '70622'
subjects = ['2/1']
#subjects = ['1/1', '1/2', '1/3', '1/4', '1/5']
#subjects = ['2/1', '2/3', '2/4', '2/6','2/7']

#recording_session = '80622'
#subjects = ['3/2', '3/3', '3/4', '3/6','3/7']
#subjects = ['4/3', '4/4', '4/5','4/6']
#subjects = ['5/1', '5/3', '5/4', '5/5','5/6']

#recording_session = '90622'
#subjects = ['6/1', '6/2', '6/3', '6/4', '6/5']
#subjects = ['7/1', '7/3', '7/4', '7/6','7/7']
#subjects = ['8/1', '8/2', '8/3', '8/4', '8/6']
#subjects = ['9/2', '9/3', '9/4', '9/5', '9/7']

#recording_session = '100622'
#subjects = ['10/1', '10/2', '10/3', '10/4', '10/5', '10/6']
#subjects = ['11/1', '11/2', '11/3', '11/4', '11/5']
#subjects = ['12/1', '12/2', '12/3']



sensors = ['optitrack', 'emg', 'TPS_calibrated']
tools_verif_tweezer = ['tweezer_x', 'tweezer_y', 'tweezer_z']
tools_verif_scissors = ['scissors_x', 'scissors_y', 'scissors_z'] 
#tools_verif_holder = ['needle_holder_x', 'needle_holder_y', 'needle_holder_z'] #subject 1
tools_verif_holder = ['needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z'] #subject 2 to 12

path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                 #lasa computer

data_segments = {}
headers = {}

for subject in subjects:
    data_segments[subject] = {}
    data_segments[subject]['tps'] = {}
    data_segments[subject]['emg'] = {}
    data_segments[subject]['optitrack'] = {}


for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION' )
    
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION', 'plot_synchro' )):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION', 'plot_synchro'))

    ####################################### get the data from the egments for synchro (one segment for each synchro movement) ###################################
    file_emg = join(data_folder, 'segment_nb_2_emg.csv')
    f_emg = open(file_emg)
    reader_emg = csv.DictReader(f_emg)
    headers['emg'] = reader_emg.fieldnames
    channels = headers['emg'][4:]
    print(channels)
            
            
    file_tps = join(data_folder, 'segment_nb_1_TPS_calibrated.csv')
    f_tps = open(file_tps)
    reader_tps = csv.DictReader(f_tps)
    headers['tps'] = reader_tps.fieldnames
    fingers_cal = headers['tps'][15:-1]
    print(fingers_cal)
            
    
    file_optitrack = join(data_folder, 'segment_nb_3_optitrack.csv')
    f_opti = open(file_optitrack)
    reader_opti = csv.DictReader(f_opti)
    #get fieldnames from DictReader object and store in list
    headers['optitrack'] = reader_opti.fieldnames
               
    tools = headers['optitrack'][3:]
    print(tools)
    
    
    ###################################### plots ########################################################
    data_emg = pd.read_csv(file_emg)
    data_tps = pd.read_csv(file_tps)
    data_optitrack = pd.read_csv(file_optitrack)
    data_segments[subject]['tps']['rel_time'] = data_tps[' relative_time']
    data_segments[subject]['emg']['rel_time'] = data_emg['relative_time']
    data_segments[subject]['optitrack']['rel_time'] = data_optitrack['relative_time']
    
    # plot TPS segment

    plt.rcParams["figure.figsize"] = (15,5)
    plt.figure()
    for finger in fingers_cal:
        data_segments[subject]['tps'][finger] = data_tps[finger]
        plt.plot(data_segments[subject]['tps']['rel_time'], data_segments[subject]['tps'][finger], label = finger)
        
    plt.xlabel('Relative time [s]')
    plt.ylabel('Force [lbs]')
    plt.title('TPS segment synchro')
    plt.legend()
    plt.grid(True)
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment1_tps.png'))
    plt.show()

    
    # Plot EMG segment
    
    plt.rcParams["figure.figsize"] = (15,5)
    plt.figure()    
    for channel in channels:
        data_segments[subject]['emg'][channel] = data_emg[channel]

    plt.plot(data_segments[subject]['emg']['rel_time'], data_segments[subject]['emg']['emg0'], label = 'emg0')
    plt.plot(data_segments[subject]['emg']['rel_time'], data_segments[subject]['emg']['emg8'], label = 'emg8')
        
    plt.xlabel('Relative time [s]')
    plt.ylabel('EMG amplitude [uV]')
    plt.title('EMG segment synchro')
    plt.ylim(-3000,3000)
    plt.grid(True)
    plt.legend()    
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment2_emg.png'))

    
    
    # Plot optitrack segment
        
    for tool in tools:
        data_segments[subject]['optitrack'][tool] = data_optitrack[tool]
        
    plt.rcParams["figure.figsize"] = (15,15)
    plt.figure()

    plt.subplot(311)
    for tool_verif in tools_verif_tweezer:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.title('Optitrack segment synchro')
    plt.subplot(312)
    for tool_verif in tools_verif_holder:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.subplot(313)
    for tool_verif in tools_verif_scissors:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment3_optitrack.png'))

        


## plot one segment (same for the 3 sensors) (need to replace manually the name of the segments)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)

recording_session = '70622'
subjects = ['2/1']
#subjects = ['1/1', '1/2', '1/3', '1/4', '1/5']
#subjects = ['2/1', '2/3', '2/4', '2/6','2/7']

#recording_session = '80622'
#subjects = ['3/2', '3/3', '3/4', '3/6','3/7']
#subjects = ['4/3', '4/4', '4/5','4/6']
#subjects = ['5/1', '5/3', '5/4', '5/5','5/6']

#recording_session = '90622'
#subjects = ['6/1', '6/2', '6/3', '6/4', '6/5']
#subjects = ['7/1', '7/3', '7/4', '7/6','7/7']
#subjects = ['8/1', '8/2', '8/3', '8/4', '8/6']
#subjects = ['9/2', '9/3', '9/4', '9/5', '9/7']

#recording_session = '100622'
#subjects = ['10/1', '10/2', '10/3', '10/4', '10/5', '10/6']
#subjects = ['11/1', '11/2', '11/3', '11/4', '11/5']
#subjects = ['12/1', '12/2', '12/3']

sensors = ['optitrack', 'emg', 'TPS_calibrated']

tools_verif_tweezer = ['tweezer_x', 'tweezer_y', 'tweezer_z']
tools_verif_scissors = ['scissors_x', 'scissors_y', 'scissors_z'] 
#tools_verif_holder = ['needle_holder_x', 'needle_holder_y', 'needle_holder_z'] #subject 1
tools_verif_holder = ['needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z'] #subject 2 to 12

path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'               

data_segments = {}
    
headers = {}

for subject in subjects:
    data_segments[subject] = {}
    data_segments[subject]['tps'] = {}
    data_segments[subject]['emg'] = {}
    data_segments[subject]['optitrack'] = {}


for subject in subjects:
    data_folder = join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION' )
    if not os.path.exists(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION', 'plot_synchro' )):
            os.makedirs(join(path_to_data_folder, recording_session , subject, 'SENSORS', 'SEGMENTATION', 'plot_synchro'))
    
    ########################################################DATA##########################################################3
    # data for emg
    
    file_emg = join(data_folder, 'segment_nb_4_emg.csv')
    f_emg = open(file_emg)
    reader_emg = csv.DictReader(f_emg)
    headers['emg'] = reader_emg.fieldnames
    channels = headers['emg'][4:]
    print(channels)
            
    # data for tps
    
    file_tps = join(data_folder, 'segment_nb_4_TPS_calibrated.csv')
    f_tps = open(file_tps)
    reader_tps = csv.DictReader(f_tps)
    headers['tps'] = reader_tps.fieldnames
    fingers_cal = headers['tps'][15:-1]
    print(fingers_cal)
    
    # data for optitrack
    
    file_optitrack = join(data_folder, 'segment_nb_4_optitrack.csv')
    f_opti = open(file_optitrack)
    reader_opti = csv.DictReader(f_opti)
    #get fieldnames from DictReader object and store in list
    headers['optitrack'] = reader_opti.fieldnames
    tools = headers['optitrack'][3:]
    print(tools)
    
    data_emg = pd.read_csv(file_emg)
    data_tps = pd.read_csv(file_tps)
    data_optitrack = pd.read_csv(file_optitrack)
    data_segments[subject]['tps']['rel_time'] = data_tps[' relative_time']
    data_segments[subject]['emg']['rel_time'] = data_emg['relative_time']
    data_segments[subject]['optitrack']['rel_time'] = data_optitrack['relative_time']
    
    ################################################## PLOTS #######################################################
    plt.rcParams["figure.figsize"] = (15,5)
    plt.figure()
    for finger in fingers_cal:
        data_segments[subject]['tps'][finger] = data_tps[finger]

        plt.plot(data_segments[subject]['tps']['rel_time'], data_segments[subject]['tps'][finger], label = finger)
        
    plt.xlabel('Relative time [s]')
    plt.ylabel('Force [lbs]')
    plt.title('TPS segment synchro')
    plt.legend()
    plt.grid(True)
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment4_tps.png'))
    plt.show()

    plt.rcParams["figure.figsize"] = (15,5)
    plt.figure()    
    for channel in channels:
        data_segments[subject]['emg'][channel] = data_emg[channel]

    plt.plot(data_segments[subject]['emg']['rel_time'], data_segments[subject]['emg']['emg0'], label = 'emg0')
    plt.plot(data_segments[subject]['emg']['rel_time'], data_segments[subject]['emg']['emg8'], label = 'emg8')
        
    plt.xlabel('Relative time [s]')
    plt.ylabel('EMG amplitude [uV]')
    plt.title('EMG segment synchro')
    plt.ylim(-3000,3000)
    plt.grid(True)
    plt.legend()    
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment4_emg.png'))

    
    for tool in tools:
        data_segments[subject]['optitrack'][tool] = data_optitrack[tool]
        
    plt.rcParams["figure.figsize"] = (15,15)
    plt.figure()

    plt.subplot(311)
    for tool_verif in tools_verif_tweezer:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.title('Optitrack segment synchro')
    plt.subplot(312)
    for tool_verif in tools_verif_holder:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.subplot(313)
    for tool_verif in tools_verif_scissors:
        plt.plot(data_segments[subject]['optitrack']['rel_time'], data_segments[subject]['optitrack'][tool_verif], label = tool_verif)
    
    plt.xlabel('Relative time [s]')
    plt.ylabel('Position [m]')
    plt.legend() 
    plt.grid(True)
    plt.savefig(join('..', 'data_exp', recording_session , subject, 'SENSORS', 'SEGMENTATION','plot_synchro', 'segment4_optitrack.png'))

        